<a href="https://colab.research.google.com/github/kalangu-asantewaa/stt_experiments_nlp/blob/main/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#installations

UsageError: %%capture is a cell magic, but the cell body is empty.


In [1]:
%%capture
!pip install datasets==1.18.3

In [ ]:
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
 !git config --global credential.helper store

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
%%capture
!apt install git-lfs

#extracting the dataset 

In [108]:
from datasets import load_dataset, load_metric, Audio

common_voice_test = load_dataset("ashesicsis1/asr-sample", "clean", split="test[0:40]")

Reusing dataset librispeech_asr (/root/.cache/huggingface/datasets/ashesicsis1___librispeech_asr/clean/2.1.0/15fb681f81615247c87e78bc9698f3598def262562594ae75a3837723c387763)


We only really need 'sentence', 'audio' and file 'path'.

In [109]:
common_voice_test = common_voice_test.remove_columns(["speaker_id", "chapter_id"])

Remove characters that don't contribute to the meaning of a word and cannot really be represented by an acoustic sound:

In [110]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_remove_regex, '', batch["text"]).lower()
    return batch

In [111]:
common_voice_test = common_voice_test.map(remove_special_characters)

0ex [00:00, ?ex/s]

#Testing the dataset 

In [112]:
#from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from jiwer import wer

##Facebook model
loading the model and the tokenizer 

In [113]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


prepared data into batches and transcribing the audio into text 

In [136]:
import numpy as np
def prepare_dataset_(batch):
    audio = batch["audio"]

    input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensors="pt" ).input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    #print(transcription[0] )
    batch["transcription"] = transcription[0].lower()
    return batch

In [ ]:
common_voice_test_ = common_voice_test.map(prepare_dataset_)#, remove_columns=common_voice_test.column_names)
transcription = np.asarray(common_voice_test_['transcription']).flatten()
#transcription.tolist()

# Word error rate (WER)

"the common_voice_test_ dataset has columns: ['file', 'audio', 'text', 'id', 'transcription']" the 'text' column is the original transcription of the text. therefore we use that as reference to the transcription generated by the model

In [140]:
print("WER:", wer(common_voice_test_['text'],transcription.tolist()))

WER: 0.042690815006468305
